<div class="alert alert-block alert-info">
<b>Number of points for this notebook:</b> 2
<br>
<b>Deadline:</b> March 17, 2021 (Wednesday) 23:00
</div>

# Exercise 3.2. Convolutional networks. VGG-style network.

In the second part you need to train a convolutional neural network with an architecture inspired by a VGG-network [(Simonyan \& Zisserman, 2015)](https://arxiv.org/abs/1409.1556).

In [1]:
skip_training = True  # Set this flag to True before validation and submission

In [2]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import tools
import tests

In [4]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is /coursedata


In [5]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [6]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## FashionMNIST dataset

Let us use the FashionMNIST dataset. It consists of 60,000 training images of 10 classes: 'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'.

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Transform to tensor
    transforms.Normalize((0.5,), (0.5,))  # Scale images to [-1, 1]
])

trainset = torchvision.datasets.FashionMNIST(root=data_dir, train=True, download=True, transform=transform)
testset = torchvision.datasets.FashionMNIST(root=data_dir, train=False, download=True, transform=transform)

classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
           'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=5, shuffle=False)

# VGG-style network

Let us now define a convolution neural network with an architecture inspired by the [VGG-net](https://arxiv.org/abs/1409.1556).

The architecture:
- A block of three convolutional layers with:
    - 3x3 kernel
    - 20 output channels
    - one pixel zero-pading on both sides
    - 2d batch normalization after each convolutional layer
    - ReLU nonlinearity after each 2d batch normalization layer
- Max pooling layer with 2x2 kernel and stride 2.
- A block of three convolutional layers with:
    - 3x3 kernel
    - 40 output channels
    - one pixel zero-pading on both sides
    - 2d batch normalization after each convolutional layer
    - ReLU nonlinearity after each 2d batch normalization layer
- Max pooling layer with 2x2 kernel and stride 2.
- One convolutional layer with:
    - 3x3 kernel
    - 60 output channels
    - *no padding*
    - 2d batch normalization after the convolutional layer
    - ReLU nonlinearity after the 2d batch normalization layer
- One convolutional layer with:
    - 1x1 kernel
    - 40 output channels
    - *no padding*
    - 2d batch normalization after the convolutional layer
    - ReLU nonlinearity after the 2d batch normalization layer
- One convolutional layer with:
    - 1x1 kernel
    - 20 output channels
    - *no padding*
    - 2d batch normalization after the convolutional layer
    - ReLU nonlinearity after the 2d batch normalization layer
- Global average pooling (compute the average value of each channel across all the input locations):
    - 5x5 kernel (the input of the layer should be 5x5)
- A fully-connected layer with 10 outputs (no nonlinearity)

Notes:
* Batch normalization is expected to be right after a convolutional layer, before nonlinearity.
* We recommend that you check the number of modules with trainable parameters in your network.

In [8]:
class VGGNet(nn.Module):
    def __init__(self):
        super(VGGNet, self).__init__()
        # YOUR CODE HERE
        self.conv1 = nn.Conv2d(1, 20, (3, 3), padding = 1)
        self.bn1 = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 20, (3, 3), padding = 1)
        self.bn2 = nn.BatchNorm2d(20)
        self.conv3 = nn.Conv2d(20, 20, (3, 3), padding = 1)
        self.bn3 = nn.BatchNorm2d(20)
        self.mp1 = nn.MaxPool2d((2, 2), stride=2)
        self.conv4 = nn.Conv2d(20, 40, (3, 3), padding = 1)
        self.bn4 = nn.BatchNorm2d(40)
        self.conv5 = nn.Conv2d(40, 40, (3, 3), padding = 1)
        self.bn5 = nn.BatchNorm2d(40)
        self.conv6 = nn.Conv2d(40, 40, (3, 3), padding = 1)
        self.bn6 = nn.BatchNorm2d(40)
        self.mp2 = nn.MaxPool2d((2, 2), stride=2)
        self.conv7 = nn.Conv2d(40, 60, (3, 3))
        self.bn7 = nn.BatchNorm2d(60)
        self.conv8 = nn.Conv2d(60, 40, (1, 1))
        self.bn8 = nn.BatchNorm2d(40)
        self.conv9 = nn.Conv2d(40, 20, (1, 1))
        self.bn9 = nn.BatchNorm2d(20)
        self.gap = nn.AvgPool2d((5, 5))
        self.fc = nn.Linear(20, 10)
        #raise NotImplementedError()

    def forward(self, x, verbose=False):
        """
        Args:
          x of shape (batch_size, 1, 28, 28): Input images.
          verbose: True if you want to print the shapes of the intermediate variables.
        
        Returns:
          y of shape (batch_size, 10): Outputs of the network.
        """
        # YOUR CODE HERE
        y = F.relu(self.bn1(self.conv1(x)))
        if verbose: print(y.shape)
        y = F.relu(self.bn2(self.conv2(y)))
        if verbose: print(y.shape)
        y = F.relu(self.bn3(self.conv3(y)))
        if verbose: print(y.shape)
        y = self.mp1(y)
        if verbose: print(y.shape)
        y = F.relu(self.bn4(self.conv4(y)))
        if verbose: print(y.shape)
        y = F.relu(self.bn5(self.conv5(y)))
        if verbose: print(y.shape)
        y = F.relu(self.bn6(self.conv6(y)))
        if verbose: print(y.shape)
        y = self.mp2(y)
        if verbose: print(y.shape)
        y = F.relu(self.bn7(self.conv7(y)))
        if verbose: print(y.shape)
        y = F.relu(self.bn8(self.conv8(y)))
        if verbose: print(y.shape)
        y = F.relu(self.bn9(self.conv9(y)))
        if verbose: print(y.shape)
        y = self.gap(y)
        if verbose: print(y.shape)
        y = y.view(-1, self.num_flat_features(y))
        if verbose: print(y.shape)
        y = self.fc(y)
        if verbose: print(y.shape)
        return y
        #raise NotImplementedError()
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [9]:
def test_VGGNet_shapes():
    net = VGGNet()
    net.to(device)

    # Feed a batch of images from the training data to test the network
    with torch.no_grad():
        images, labels = iter(trainloader).next()
        images = images.to(device)
        print('Shape of the input tensor:', images.shape)

        y = net(images, verbose=True)
        assert y.shape == torch.Size([trainloader.batch_size, 10]), f"Bad y.shape: {y.shape}"

    print('Success')

test_VGGNet_shapes()

Shape of the input tensor: torch.Size([32, 1, 28, 28])
torch.Size([32, 20, 28, 28])
torch.Size([32, 20, 28, 28])
torch.Size([32, 20, 28, 28])
torch.Size([32, 20, 14, 14])
torch.Size([32, 40, 14, 14])
torch.Size([32, 40, 14, 14])
torch.Size([32, 40, 14, 14])
torch.Size([32, 40, 7, 7])
torch.Size([32, 60, 5, 5])
torch.Size([32, 40, 5, 5])
torch.Size([32, 20, 5, 5])
torch.Size([32, 20, 1, 1])
torch.Size([32, 20])
torch.Size([32, 10])
Success


In [10]:
tests.test_vgg_net(VGGNet)

y: tensor([[ 10.0382,  10.0382,  10.0382,  10.0382,  10.0382, -10.0382, -10.0382,
         -10.0382, -10.0382, -10.0382]], grad_fn=<AddmmBackward>)
expected: tensor([ 10.0382,  10.0382,  10.0382,  10.0382,  10.0382, -10.0382, -10.0382,
        -10.0382, -10.0382, -10.0382])
Success


# Train the network

In [11]:
# This function computes the accuracy on the test dataset
def compute_accuracy(net, testloader):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

### Training loop

Your task is to implement the training loop. The recommended hyperparameters:
* Adam optimizer with learning rate 0.01.
* Cross-entropy loss. Note that we did not use softmax nonlinearity in the final layer of our network. Therefore, we need to use a loss function with log_softmax implemented, such as [`nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss).
* Number of epochs: 10

We recommend you to use function `compute_accuracy()` defined above to track the accaracy during training. The test accuracy should be above 0.89.

**Note: function `compute_accuracy()` sets the network into the evaluation mode which changes the way the batch statistics are computed in batch normalization. You need to set the network into the training mode (by calling `net.train()`) when you want to perform training.**

In [12]:
net = VGGNet()

In [13]:
# Implement the training loop in this cell
if not skip_training:
    # YOUR CODE HERE
    net.zero_grad()
    lossFunc = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
    N = 10
    for _ in range(N):
        net.train()
        for images, labels in trainloader:   
            optimizer.zero_grad()
            y = net(images)
            loss = lossFunc(y, labels)
            loss.backward()
            optimizer.step()
        
        accuracy = compute_accuracy(net,testloader)
        print(accuracy)
    #raise NotImplementedError()

In [14]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
# Set confirm=False if you do not want to be asked for confirmation before saving.
if not skip_training:
    tools.save_model(net, '2_vgg_net.pth', confirm=True)

In [15]:
if skip_training:
    net = VGGNet()
    tools.load_model(net, '2_vgg_net.pth', device)

Model loaded from 2_vgg_net.pth.


In [16]:
# Compute the accuracy on the test set
accuracy = compute_accuracy(net, testloader)
print(f'Accuracy of the VGG net on the test images: {accuracy: .3f}')
assert accuracy > 0.89, 'Poor accuracy'
print('Success')

Accuracy of the VGG net on the test images:  0.923
Success
